In [ ]:
import socket
import threading
import json

class SequencerServer:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.server_socket = None
        self.clients = []
        self.sequence_number = 0  # For ordering messages

    def start(self):
        try:
            self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.server_socket.bind((self.host, self.port))  # Updated host
            self.server_socket.listen(5)
            print(f"Sequencer Server running on {self.host}:{self.port}...")
            
            while True:
                client_socket, client_address = self.server_socket.accept()
                self.clients.append(client_socket)
                print(f"New connection from {client_address}")
                threading.Thread(target=self.handle_client, args=(client_socket,)).start()
        except OSError as e:
            print(f"Error starting server: {e}")
        finally:
            if self.server_socket:
                self.server_socket.close()

    def handle_client(self, client_socket):
        try:
            while True:
                data = client_socket.recv(1024).decode()
                if not data:
                    break
                message = json.loads(data)

                # Assign a sequence number
                self.sequence_number += 1
                message['sequence'] = self.sequence_number
                print(f"Sequenced Message: {message}")

                # Broadcast message to all clients
                self.broadcast_message(message)
        except Exception as e:
            print(f"Error with client: {e}")
        finally:
            self.clients.remove(client_socket)
            client_socket.close()

    def broadcast_message(self, message):
        for client in self.clients:
            try:
                client.sendall(json.dumps(message).encode())
            except Exception as e:
                print(f"Error broadcasting to a client: {e}")

if __name__ == "__main__":
    # Use your computer's IP or bind to all interfaces
    server = SequencerServer("192.168.27.221", 8080)  # Listen on all network interfaces

    try:
        server.start()
    except KeyboardInterrupt:
        print("Server shutting down.")


Sequencer Server running on 0.0.0.0:8080...
